In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=ecb5b458395b44b951adfe793672ca0119ffdf1bd55df02966c0bade390c2463
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 39.6 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [ ]:
 # create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
import pyspark
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [ ]:
from google.colab import files
files.upload()

Saving people.json to people.json


{'people.json': b'{"name":"Michael"}\n{"name":"Andy", "age":30}\n{"name":"Justin", "age":19}\n{"name":"Mary", "age":29}\n{"name":"John"}\n{"name":"Juhi", "age":36}\n{"name":"Jasmeen", "age":13}\n{"name":"Seeta", "age":12}\n{"age":45}\n{"name":"Mohan", "age":34}\n{"name":"Kayen", "age":18}\n{"age":67}\n{"name":"Caley", "age":21}\n{"name":"Shyam", "age":19}\n'}

In [7]:
df = spark.read.json("people.json")
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [9]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|NULL|Michael|
|  30|   Andy|
|  19| Justin|
|  29|   Mary|
|NULL|   John|
|  36|   Juhi|
|  13|Jasmeen|
|  12|  Seeta|
|  45|   NULL|
|  34|  Mohan|
|  18|  Kayen|
|  67|   NULL|
|  21|  Caley|
|  19|  Shyam|
+----+-------+



1. Count number of records


In [8]:
df.count()

14

2. create a new column with the simple copy of age column.


In [11]:
from pyspark.sql.functions import col

df = df.withColumn("new_age",col("age"))
df.show()

+----+-------+-------+
| age|   name|new_age|
+----+-------+-------+
|NULL|Michael|   NULL|
|  30|   Andy|     30|
|  19| Justin|     19|
|  29|   Mary|     29|
|NULL|   John|   NULL|
|  36|   Juhi|     36|
|  13|Jasmeen|     13|
|  12|  Seeta|     12|
|  45|   NULL|     45|
|  34|  Mohan|     34|
|  18|  Kayen|     18|
|  67|   NULL|     67|
|  21|  Caley|     21|
|  19|  Shyam|     19|
+----+-------+-------+



3. create a new column containing ages of people one year bigger.


In [18]:
from pyspark.sql.functions import expr

df = df.withColumn("one_plus_age",expr("age + 1"))
df.show()

+----+-------+-------+------------+
| age|   name|new_age|one_plus_age|
+----+-------+-------+------------+
|NULL|Michael|   NULL|        NULL|
|  30|   Andy|     30|          31|
|  19| Justin|     19|          20|
|  29|   Mary|     29|          30|
|NULL|   John|   NULL|        NULL|
|  36|   Juhi|     36|          37|
|  13|Jasmeen|     13|          14|
|  12|  Seeta|     12|          13|
|  45|   NULL|     45|          46|
|  34|  Mohan|     34|          35|
|  18|  Kayen|     18|          19|
|  67|   NULL|     67|          68|
|  21|  Caley|     21|          22|
|  19|  Shyam|     19|          20|
+----+-------+-------+------------+



4. Show the records where age column is missing.

In [19]:
df.filter(col("age").isNull()).show()

+----+-------+-------+------------+
| age|   name|new_age|one_plus_age|
+----+-------+-------+------------+
|NULL|Michael|   NULL|        NULL|
|NULL|   John|   NULL|        NULL|
+----+-------+-------+------------+



5. Fetch the records having atleast one non-null values.

In [20]:
df.na.drop(how = 'all').show()

+----+-------+-------+------------+
| age|   name|new_age|one_plus_age|
+----+-------+-------+------------+
|NULL|Michael|   NULL|        NULL|
|  30|   Andy|     30|          31|
|  19| Justin|     19|          20|
|  29|   Mary|     29|          30|
|NULL|   John|   NULL|        NULL|
|  36|   Juhi|     36|          37|
|  13|Jasmeen|     13|          14|
|  12|  Seeta|     12|          13|
|  45|   NULL|     45|          46|
|  34|  Mohan|     34|          35|
|  18|  Kayen|     18|          19|
|  67|   NULL|     67|          68|
|  21|  Caley|     21|          22|
|  19|  Shyam|     19|          20|
+----+-------+-------+------------+



6. Fill the missing values of age column with mean.


In [22]:
from pyspark.sql.functions import mean

mean_age = df.select(mean(col("age"))).collect()[0][0]

df = df.fillna(mean_age,subset = ['age'])
df.show()

+---+-------+-------+------------+
|age|   name|new_age|one_plus_age|
+---+-------+-------+------------+
| 28|Michael|   NULL|        NULL|
| 30|   Andy|     30|          31|
| 19| Justin|     19|          20|
| 29|   Mary|     29|          30|
| 28|   John|   NULL|        NULL|
| 36|   Juhi|     36|          37|
| 13|Jasmeen|     13|          14|
| 12|  Seeta|     12|          13|
| 45|   NULL|     45|          46|
| 34|  Mohan|     34|          35|
| 18|  Kayen|     18|          19|
| 67|   NULL|     67|          68|
| 21|  Caley|     21|          22|
| 19|  Shyam|     19|          20|
+---+-------+-------+------------+



7. Write a sql query to find the people with age greater than 19.


In [25]:
df.createOrReplaceTempView("people")
spark.sql("SELECT * FROM people WHERE age > 19 ").show()

+---+-------+-------+------------+
|age|   name|new_age|one_plus_age|
+---+-------+-------+------------+
| 28|Michael|   NULL|        NULL|
| 30|   Andy|     30|          31|
| 29|   Mary|     29|          30|
| 28|   John|   NULL|        NULL|
| 36|   Juhi|     36|          37|
| 45|   NULL|     45|          46|
| 34|  Mohan|     34|          35|
| 67|   NULL|     67|          68|
| 21|  Caley|     21|          22|
+---+-------+-------+------------+

